In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.rdd import portable_hash
from pyspark.statcounter import StatCounter
from pyspark.sql.types import StringType,IntegerType,FloatType
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp, current_timestamp, col,expr,unix_timestamp,round,when,hour
import os
import json
%matplotlib inline
from datetime import datetime
from matplotlib import pyplot as plt
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler, VectorSlicer
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit , CrossValidator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import IndexToString

In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FAF").getOrCreate()
sc=spark.sparkContext
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df_tons_train_17 = spark.read.csv("/FileStore/tables/df_tons_train_17.csv",header=True, inferSchema=True)
df_tons_test_17 = spark.read.csv("/FileStore/tables/df_tons_test_17.csv",header=True, inferSchema=True)
df_tons_predicted_17 = spark.read.csv("/FileStore/tables/df_tons_predicted_17.csv",header=True, inferSchema=True)


In [0]:
dict_1 = {11 :"O_Birmingham-Hoover-Talladega, O_AL CFS Area" , 12 : "O_Mobile-Daphne-Fairhope, O_AL CFS Area" , 19 : "O_Remainder of Alabama" , 20 : "O_Alaska" , 41 : "O_Phoenix-Mesa-Glendale, O_AZ CFS Area" , 42 : "O_Tucson-Nogales, AZ CFS Area" , 49 : "O_Remainder of Arizona" , 50 : "O_Remainder of Arkansas" , 61 : "O_Los Angeles-Long Beach, CA CFS Area" , 62 : "O_Sacramento-Roseville, CA CFS Area" , 63 : "O_San Diego-Carlsbad-San Marcos, CA CFS Area" , 64 : "O_San Jose-San Francisco-Oakland, CA CFS Area" , 65 : "O_Fresno-Madera, CA CFS Area" , 69 : "O_Remainder of California" , 81 : "O_Denver-Aurora, CO CFS Area" , 89 : "O_Remainder of Colorado" , 91 : "O_Hartford-West Hartford-East Hartford, CT CFS Area" , 92 : "O_New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)" , 99 : "O_Remainder of Connecticut" , 101 : "O_Philadelphia-Reading-Camden, PA-NJ-DE-MD CFS Area (DE Part)" , 109 : "O_Remainder of Delaware" , 111 : "O_Washington-Arlington-Alexandria, DC-VA-MD-WV CFS Area (DC Part)" , 121 : "O_Jacksonville-St. Marys-Palatka, FL-GA CFS Area (FL Part)" , 122 : "O_Miami-Fort Lauderdale-Port St. Lucie, FL CFS Area" , 123 : "O_Orlando-Deltona-Daytona Beach, FL CFS Area" , 124 : "O_Tampa-St. Petersburg-Clearwater, FL CFS Area" , 129 : "O_Remainder of Florida" , 131 : "O_Atlanta-Athens-Clarke County-Sandy Springs, GA CFS Area" , 132 : "O_Savannah-Hinesville-Statesboro, GA CFS Area" , 139 : "O_Remainder of Georgia" , 151 : "O_Urban Honolulu, HI CFS Area" , 159 : "O_Remainder of Hawaii" , 160 : "O_Idaho" , 171 : "O_Chicago-Naperville, IL-IN-WI CFS Area (IL Part)" , 172 : "O_St. Louis-St. Charles-Farmington, MO-IL CFS Area(IL Part)" , 179 : "O_Remainder of Illinois" , 181 : "O_Chicago-Naperville, IL-IN-WI CFS Area (IN Part)" , 182 : "O_Indianapolis-Carmel-Muncie, IN CFS Area" , 183 : "O_Fort Wayne-Huntington-Auburn, IN CFS Area" , 189 : "O_Remainder of Indiana" , 190 : "O_Remainder of Iowa" , 201 : "O_Kansas City-Overland Park-Kansas City, MO-KS CFS Area (KS Part)" , 202 : "O_Wichita-Arkansas City-Winfield, KS CFS Area" , 209 : "O_Remainder of Kansas" , 211 : "O_Cincinnati-Wilmington-Maysville, OH-KY-IN CFS Area (KY Part)" , 212 : "O_Louisville/Jefferson County-Elizabethtown-Madison, KY-IN CFS Area (KY Part)" , 219 : "O_Remainder of Kentucky" , 221 : "O_Baton Rouge, LA CFS Area" , 222 : "O_Lake Charles, LA CFS Area" , 223 : "O_New Orleans-Metairie-Hammond, LA-MS CFS Area(LA Part)" , 229 : "O_Remainder of Louisiana" , 230 : "O_Remainder of Maine" , 241 : "O_Baltimore-Columbia-Towson, MD CFS Area" , 242 : "O_Washington-Arlington-Alexandria, DC-VA-MD-WV CFS Area (MD Part)" , 249 : "O_Remainder of Maryland" , 251 : "O_Boston-Worcester-Providence, MA-RI-NH-CT CFS Area (MA Part)" , 259 : "O_Remainder of Massachusetts" , 261 : "O_Detroit-Warren-Ann Arbor, MI CFS Area" , 262 : "O_Grand Rapids-Wyoming-Muskegon, MI CFS Area" , 269 : "O_Remainder of Michigan" , 271 : "O_Minneapolis-St. Paul, MN-WI CFS Area (MN Part)" , 279 : "O_Remainder of Minnesota" , 280 : "O_Remainder of Mississippi" , 291 : "O_Kansas City-Overland Park-Kansas City, MO-KS CFS Area (MO Part)" , 292 : "O_St. Louis-St. Charles-Farmington, MO-IL CFS Area(MO Part)" , 299 : "O_Remainder of Missouri" , 300 : "O_Remainder of Montana" , 311 : "O_Omaha-Council Bluffs-Fremont, NE-IA CFS Area(NE Part)" , 319 : "O_Remainder of Nebraska" , 321 : "O_Las Vegas-Henderson, NV-AZ CFS Area (NV Part)" , 329 : "O_Remainder of Nevada" , 331 : "O_Boston-Worcester-Providence, MA-RI-NH-CT CFS Area (NH Part)" , 339 : "O_Remainder of New Hampshire" , 341 : "O_New York-Newark, NY-NJ-CT-PA CFS Area (NJ Part)" , 342 : "O_Philadelphia-Reading-Camden, PA-NJ-DE-MD CFS Area (NJ Part)" , 350 : "O_Remainder of New Mexico" , 361 : "O_Albany-Schenectady, NY CFS Area" , 362 : "O_Buffalo-Cheektowaga, NY CFS Area" , 363 : "O_New York-Newark, NY-NJ-CT-PA CFS Area (NYPart)" , 364 : "O_Rochester-Batavia-Seneca Falls, NY CFS Area" , 369 : "O_Remainder of New York" , 371 : "O_Charlotte-Concord, NC-SC CFS Area (NC Part)" , 372 : "O_Greensboro-Winston-Salem-High Point, NC CFS Area" , 373 : "O_Raleigh-Durham-Chapel Hill, NC CFS Area" , 379 : "O_Remainder of North Carolina" , 380 : "O_Remainder of North Dakota" , 391 : "O_Cincinnati-Wilmington-Maysville, OH-KY-IN CFS Area (OH Part)" , 392 : "O_Cleveland-Akron-Canton, OH CFS Area" , 393 : "O_Columbus-Marion-Zanesville, OH CFS Area" , 394 : "O_Dayton-Springfield-Sidney, OH CFS Area" , 399 : "O_Remainder of Ohio" , 401 : "O_Oklahoma City-Shawnee, OK CFS Area" , 402 : "O_Tulsa-Muskogee-Bartlesville, OK CFS Area" , 409 : "O_Remainder of Oklahoma" , 411 : "O_Portland-Vancouver-Salem, OR-WA CFS Area (OR Part)" , 419 : "O_Remainder of Oregon" , 421 : "O_Philadelphia-Reading-Camden, PA-NJ-DE-MD CFS Area (PA Part)" , 422 : "O_Pittsburgh-New Castle-Weirton, PA-OH-WV CFS Area (PA Part)" , 423 : "O_New York-Newark, NY-NJ-CT-PA CFS Area (PA Part)" , 429 : "O_Remainder of Pennsylvania" , 441 : "O_Boston-Worcester-Providence, MA-RI-NH-CT CFS Area (RI Part)" , 451 : "O_Charleston-North Charleston-Summerville, SC CFS Area" , 452 : "O_Greenville-Spartanburg-Anderson, SC CFS Area" , 459 : "O_Remainder of South Carolina" , 460 : "O_Remainder of South Dakota" , 471 : "O_Memphis, TN-MS-AR CFS Area (TN Part)" , 472 : "O_Nashville-Davidson-Murfreesboro, TN CFS Area" , 473 : "O_Knoxville-Morristown-Sevierville, TN CFS Area" , 479 : "O_Remainder of Tennessee" , 481 : "O_Austin-Round Rock, TX CFS Area" , 482 : "O_Beaumont-Port Arthur, TX CFS Area" , 483 : "O_Corpus Christi-Kingsville-Alice, TX CFS Area" , 484 : "O_Dallas-Fort Worth, TX-OK CFS Area (TX Part)" , 485 : "O_El Paso-Las Cruces, TX-NM CFS Area (TX Part)" , 486 : "O_Houston-The Woodlands, TX CFS Area" , 487 : "O_Laredo, TX CFS Area" , 488 : "O_San Antonio-New Braunfels, TX CFS Area" , 489 : "O_Remainder of Texas" , 491 : "O_Salt Lake City-Provo-Orem, UT CFS Area" , 499 : "O_Remainder of Utah" , 500 : "O_Remainder of Vermont" , 511 : "O_Richmond, VA CFS Area" , 512 : "O_Virginia Beach-Norfolk, VA-NC CFS Area (VA Part)" , 513 : "O_Washington-Arlington-Alexandria, DC-VA-MD-WV CFS Area (VA Part)" , 519 : "O_Remainder of Virginia" , 531 : "O_Seattle-Tacoma, WA CFS Area" , 532 : "O_Portland-Vancouver-Salem, OR-WA CFS Area (WA Part)" , 539 : "O_Remainder of Washington" , 540 : "O_Remainder of West Virginia" , 551 : "O_Milwaukee-Racine-Waukesha, WI CFS Area" , 559 : "O_Remainder of Wisconsin" , 560 : "O_Remainder of Wyoming"}

In [0]:
user_func =  udf(lambda x: dict_1.get(x), StringType())
user_func_1 =  udf(lambda x: dict_1.get(x), StringType())

df_tons_train_17 = df_tons_train_17.withColumn('dms_orig',user_func(df_tons_train_17.dms_orig))
df_tons_train_17 = df_tons_train_17.withColumn('dms_dest',user_func_1(df_tons_train_17.dms_dest))

df_tons_test_17 = df_tons_test_17.withColumn('dms_orig',user_func(df_tons_test_17.dms_orig))
df_tons_test_17 = df_tons_test_17.withColumn('dms_dest',user_func_1(df_tons_test_17.dms_dest))


df_tons_predicted_17 = df_tons_predicted_17.withColumn('dms_orig',user_func(df_tons_predicted_17.dms_orig))
df_tons_predicted_17 = df_tons_predicted_17.withColumn('dms_dest',user_func_1(df_tons_predicted_17.dms_dest))

In [0]:
Commodity_dic={1 : "Animals and Fish (live)" , 2 : "Cereal Grains (includes seed)" , 3 : "Agricultural Products (excludes Animal Feed, Cereal Grains,Forage Products)" , 4 : "Animal Feed, Eggs, Honey, Other Products of Animal Origin" , 5 : "Meat, Poultry, Fish, Seafood, and Their Preparations" , 6 : "Milled Grain Products and Preparations, and Bakery Products" , 7 : "Other Prepared Foodstuffs, Fats and Oils" , 8 : "Alcoholic Beverages and Denatured Alcohol" , 9 : "Tobacco Products" , 10 : "Monumental or Building Stone" , 11 : "Natural Sands" , 12 : "Gravel and Crushed Stone (excludes Dolomite and Slate)" , 13 : "Other Non-Metallic Minerals not elsewhere classified" , 14 : "Metallic Ores and Concentrates" , 15 : "Coal" , 16 : "Crude Petroleum" , 17 : "Gasoline, Aviation Turbine Fuel, and Ethanol (includes Kerosene, and Fuel Alcohols)" , 18 : "Fuel Oils (includes Diesel, Bunker C, and Biodiesel)" , 19 : "Other Coal and Petroleum Products, not elsewhere classified" , 20 : "Basic Chemicals" , 21 : "Pharmaceutical Products" , 22 : "Fertilizers" , 23 : "Other Chemical Products and Preparations" , 24 : "Plastics and Rubber" , 25 : "Logs and Other Wood in the Rough" , 26 : "Wood Products" ,  27 : "Pulp, Newsprint, Paper, and Paperboard" , 28 : "Paper or Paperboard Articles" , 29 : "Printed Products" , 30 : "Textiles, Leather, and Articles of Textiles or Leather" , 31 : "Non-Metallic Mineral Products" , 32 : "Base Metal in Primary or Semi-Finished Forms and in Finished Basic Shapes" , 33 : "Articles of Base Metal" , 34 : "Machinery" , 35 : "Electronic and Other Electrical Equipment and Components, and Office Equipment" , 36 : "Motorized and Other Vehicles (includes parts)" , 37 : "Transportation Equipment, not elsewhere classified" ,  38 : "Precision Instruments and Apparatus" , 39 : "Furniture, Mattresses and Mattress Supports, Lamps, Lighting Fittings, and Illuminated Signs" , 40 : "Miscellaneous Manufactured Products" , 41 : "Waste and Scrap (excludes of agriculture or food)" , 43 : "Mixed Freight" , 99 : "Commodity unknown"}

In [0]:
com_user_func =  udf(lambda x: Commodity_dic.get(x), StringType())
df_tons_train_17 = df_tons_train_17.withColumn('sctg2',com_user_func(df_tons_train_17.sctg2))
df_tons_test_17 = df_tons_test_17.withColumn('sctg2',com_user_func(df_tons_test_17.sctg2))
df_tons_predicted_17 = df_tons_predicted_17.withColumn('sctg2',com_user_func(df_tons_predicted_17.sctg2))

In [0]:
Mode_dic={1 : "Truck" , 2 : "Rail" , 3 : "Water" , 4 : "Air (includes truck-air)" , 5 : "Multiple Modes and Mail" , 6 : "Pipeline" , 7 : "Other and Unknown" , 8 : "No Domestic Mode"}

In [0]:
Mode_user_func =  udf(lambda x: Mode_dic.get(x), StringType())

df_tons_train_17 = df_tons_train_17.withColumn('dms_mode',Mode_user_func(df_tons_train_17.dms_mode))
df_tons_test_17 = df_tons_test_17.withColumn('dms_mode',Mode_user_func(df_tons_test_17.dms_mode))
df_tons_predicted_17 = df_tons_predicted_17.withColumn('dms_mode',Mode_user_func(df_tons_predicted_17.dms_mode))

In [0]:
encoding_var = [i[0] for i in df_tons_train_17.dtypes if (i[1]=='string') & (i[0]!='tons') & (i[0]!='dms_orig')]
print("encoding_var:",encoding_var)

num_var = [i[0] for i in df_tons_train_17.dtypes if ((i[1]=='int') | (i[1]=='double') | (i[1]=='vector')) & (i[0]!='tons') & (i[0]!='year') & (i[0]!='wgt_dist')]
print("num_var:",num_var)

encoding_var: ['dms_dest', 'dms_mode', 'sctg2']
num_var: ['tons_GrowthRate']

In [0]:
assemblers_scaled = [VectorAssembler(inputCols=[col], outputCol=col + "_vecc") for col in num_var]
scalers = [MinMaxScaler(inputCol=col + "_vecc", outputCol=col + "_scaled") for col in num_var]
string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
assembler = VectorAssembler(inputCols = [col + "_scaled" for col in num_var] + ['OHE_' + c for c in encoding_var], outputCol = "features")

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = "features", labelCol="tons",maxIter=100)
pipeline = Pipeline(stages=assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler,lr ])

In [0]:
lr1 = LinearRegression()

print(lr1.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [0]:
model_lr_17=pipeline.fit(df_tons_train_17)

In [0]:
predictions_test_lr_17 = model_lr_17.transform(df_tons_test_17)
predictions_test_lr_17.select("features","tons","prediction").show(5)

+--------------------+-------+-------------------+
 features| tons| prediction|
+--------------------+-------+-------------------+
(181,[0,2,134,143...| 6.9566|-3.1090835244455803|
(181,[0,9,133,176...|177.098| 1136.8895020950933|
(181,[0,65,133,14...| 0.0508| 138.89658962111363|
(181,[0,88,134,14...| 0.0516| -18.3369167584598|
(181,[0,77,133,15...| 0.0351| 48.51442059869586|
+--------------------+-------+-------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_test_lr_17)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_test_lr_17)
print("R squared (R2) on test data = %g" % r2)

#n = predictions_test_c1.count()
n=predictions_test_lr_17.count()
p = 181
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
print("adjusted_r2 on test data = %g" % adjusted_r2)

Root Mean Squared Error (RMSE) on test data = 397.266
R squared (R2) on test data = 0.198657
adjusted_r2 on test data = 0.190484

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor
Dt = DecisionTreeRegressor(featuresCol = "features", labelCol="tons",maxDepth=30,maxBins=17)
pipeline = Pipeline(stages=assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler,Dt ])

In [0]:
Dt1 = DecisionTreeRegressor()

print(Dt1.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -1407754390808368278)
varianceCol: column name for the biased sample variance of prediction. (undefined)

In [0]:
model_dt_17=pipeline.fit(df_tons_train_17)

In [0]:
predictions_test_dt_17 = model_dt_17.transform(df_tons_test_17)
predictions_test_dt_17.select("features","tons","prediction").show(5)

+--------------------+-------+-------------------+
 features| tons| prediction|
+--------------------+-------+-------------------+
(181,[0,2,134,143...| 6.9566|0.18632641272336384|
(181,[0,9,133,176...|177.098| 174.62840000000003|
(181,[0,65,133,14...| 0.0508| 16.168670222222225|
(181,[0,88,134,14...| 0.0516|0.18632641272336384|
(181,[0,77,133,15...| 0.0351| 14.614688654545446|
+--------------------+-------+-------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_test_dt_17)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_test_dt_17)
print("R squared (R2) on test data = %g" % r2)

n=predictions_test_dt_17.count()
p = 181
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
print("adjusted_r2 on test data = %g" % adjusted_r2)

Root Mean Squared Error (RMSE) on test data = 153.163
R squared (R2) on test data = 0.880886
adjusted_r2 on test data = 0.879671

In [0]:
from pyspark.ml.regression import GBTRegressor
GBT = GBTRegressor(featuresCol = "features", labelCol="tons",maxIter=5,maxDepth=30,maxBins=17)
pipeline = Pipeline(stages=assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler,GBT ])

In [0]:
GBT1=GBTRegressor()
print(GBT1.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: all)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: squared, absolute (default: squared)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxIter: max number of iterations (>= 0). (default: 20)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -6682481135904123338)
stepSize: Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator. (default: 0.1)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)

In [0]:
model_gbt_17=pipeline.fit(df_tons_train_17)

In [0]:
predictions_test_gbt_17 = model_gbt_17.transform(df_tons_test_17)
predictions_test_gbt_17.select("features","tons","prediction").show(5)

+--------------------+-------+-------------------+
 features| tons| prediction|
+--------------------+-------+-------------------+
(181,[0,2,134,143...| 6.9566| 0.7091408461170998|
(181,[0,9,133,176...|177.098| 174.29526718029015|
(181,[0,65,133,14...| 0.0508| 17.534445861842666|
(181,[0,88,134,14...| 0.0516|0.15964435252843628|
(181,[0,77,133,15...| 0.0351| 13.266779263550058|
+--------------------+-------+-------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_test_gbt_17)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_test_gbt_17)
print("R squared (R2) on test data = %g" % r2)

n=predictions_test_gbt_17.count()
p = 181
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
print("adjusted_r2 on test data = %g" % adjusted_r2)

Root Mean Squared Error (RMSE) on test data = 205.742
R squared (R2) on test data = 0.785068
adjusted_r2 on test data = 0.782876

In [0]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol = "features", labelCol="tons",numTrees=5,maxDepth=30,maxBins=17)
pipeline = Pipeline(stages=assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler,rf ])

In [0]:
RF1=RandomForestRegressor()
print(RF1.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
numTrees: Number of trees to train (>= 1). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 2502083311556356884)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)

In [0]:
model_rf_17=pipeline.fit(df_tons_train_17)

In [0]:
predictions_test_rf_17 = model_rf_17.transform(df_tons_test_17)
predictions_test_rf_17.select("features","tons","prediction").show(5)

+--------------------+-------+------------------+
 features| tons| prediction|
+--------------------+-------+------------------+
(181,[0,2,134,143...| 6.9566|0.7745167914011182|
(181,[0,9,133,176...|177.098|123.61925800000002|
(181,[0,65,133,14...| 0.0508|32.835637969288776|
(181,[0,88,134,14...| 0.0516|0.2608721637484396|
(181,[0,77,133,15...| 0.0351|13.095625501117746|
+--------------------+-------+------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_test_rf_17)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_test_rf_17)
print("R squared (R2) on test data = %g" % r2)

n=predictions_test_rf_17.count()
p = 181
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
print("adjusted_r2 on test data = %g" % adjusted_r2)

Root Mean Squared Error (RMSE) on test data = 180.058
R squared (R2) on test data = 0.83538
adjusted_r2 on test data = 0.833701

In [0]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(featuresCol = "features", labelCol="tons",maxIter=100,family="gaussian")
pipeline = Pipeline(stages=assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler,glr ])

In [0]:
glr1=GeneralizedLinearRegression()
print(glr1.explainParams())

family: The name of family which is a description of the error distribution to be used in the model. Supported options: gaussian (default), binomial, poisson, gamma and tweedie. (default: gaussian)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
link: The name of link function which provides the relationship between the linear predictor and the mean of the distribution function. Supported options: identity, log, inverse, logit, probit, cloglog and sqrt. (undefined)
linkPower: The index in the power link function. Only applicable to the Tweedie family. (undefined)
linkPredictionCol: link prediction (linear predictor) column name (undefined)
maxIter: max number of iterations (>= 0). (default: 25)
offsetCol: The offset column name. If this is not set or empty, we treat all instance offsets as 0.0 (undefined)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: irls. (default: irls)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
variancePower: The power in the variance function of the Tweedie distribution which characterizes the relationship between the variance and mean of the distribution. Only applicable for the Tweedie family. Supported values: 0 and [1, Inf). (default: 0.0)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [0]:
model_glr_17=pipeline.fit(df_tons_train_17)

In [0]:
predictions_test_glr_17 = model_glr_17.transform(df_tons_test_17)
predictions_test_glr_17.select("features","tons","prediction").show(5)

+--------------------+-------+------------------+
 features| tons| prediction|
+--------------------+-------+------------------+
(182,[0,1,3,135,1...| 6.9566| 18.35818390172716|
(182,[0,1,10,134,...|177.098|1137.4401578618474|
(182,[0,1,65,134,...| 0.0508|118.30279125745955|
(182,[0,1,89,135,...| 0.0516| -34.9878392331324|
(182,[0,1,78,134,...| 0.0351| 32.88090868262646|
+--------------------+-------+------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_test_glr_17)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(labelCol="tons", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_test_glr_17)
print("R squared (R2) on test data = %g" % r2)

n=predictions_test_glr_17.count()
p = 181
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
print("adjusted_r2 on test data = %g" % adjusted_r2)

Root Mean Squared Error (RMSE) on test data = 397.053
R squared (R2) on test data = 0.199515
adjusted_r2 on test data = 0.191306